In [102]:
import pandas as pd
import polars as pl
import numpy as np
import scipy
import os
from Bio import SeqIO, SearchIO
from BCBio import GFF
import math
from tqdm.auto import tqdm
# import torch
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import StandardScaler
from Bio.SeqFeature import SeqFeature, FeatureLocation
from scipy.io import mmwrite
from collections import defaultdict

In [103]:
genome_path = "/large_storage/hielab/changdan/euk_genomes/GCF_009914755.1_T2T-CHM13v2.0_genomic.gbff"
activations_path = "/large_storage/hielab/changdan/evo2_T2T_activations_sae-a6u40nnl-layer26-batch-topk-tied-expansion_8-k_64/"

In [104]:
! ls ../data

chm13v2.0_rmsk.bb      README.md			      script.sh
hs1.repeatMasker.gff3  rm2gff3.sh
hs1.repeatMasker.out   sae-layer26-mixed-expansion_8-k_64.pt


In [105]:
column_names = [
    'seqid',      # Sequence/chromosome name
    'source',     # Source of the annotation
    'type',       # Feature type
    'start',      # Start position (1-based)
    'end',        # End position (inclusive)
    'score',      # Score (or '.')
    'strand',     # Strand (+ or - or .)
    'phase',      # Phase (0, 1, 2, or .)
    'attributes'  # Semicolon-separated attributes
]

repeat_masker = pl.read_csv("../data/hs1.repeatMasker.gff3", separator="\t", comment_prefix="#", has_header=False)
repeat_masker.columns = column_names
repeat_masker

seqid,source,type,start,end,score,strand,phase,attributes
str,str,str,i64,i64,i64,str,str,str
"""chrM""","""RepeatMasker-4.0.1""","""similarity""",2014,2169,311,"""+""",""".""","""Target=LSU-rRNA_Hsa rRNA 3753 …"
"""chrM""","""RepeatMasker-4.0.1""","""similarity""",2653,2730,650,"""+""",""".""","""Target=tRNA-Leu-TTA(m) tRNA 1 …"
"""chrM""","""RepeatMasker-4.0.1""","""similarity""",3752,3823,429,"""-""",""".""","""Target=tRNA-Gln-CAA_ tRNA 1 72…"
"""chrM""","""RepeatMasker-4.0.1""","""similarity""",6871,6937,432,"""-""",""".""","""Target=tRNA-Ser-TCA(m) tRNA 1 …"
"""chrM""","""RepeatMasker-4.0.1""","""similarity""",15628,15672,13,"""+""",""".""","""Target=(CAACTAT)n Simple_repea…"
…,…,…,…,…,…,…,…,…
"""chrY""","""RepeatMasker-4.0.1""","""similarity""",62448441,62448531,307,"""+""",""".""","""Target=MIR3 SINE/MIR 49 143;Di…"
"""chrY""","""RepeatMasker-4.0.1""","""similarity""",62452004,62452106,256,"""+""",""".""","""Target=MER5B DNA/hAT-Charlie 1…"
"""chrY""","""RepeatMasker-4.0.1""","""similarity""",62452109,62452299,527,"""+""",""".""","""Target=L1MC5a LINE/L1 5452 567…"


In [106]:
Counter(repeat_masker['strand'])

Counter({'+': 3206651, '-': 2384428})

In [107]:
repeat_masker['attributes'][4]

'Target=(CAACTAT)n Simple_repeat 1 47;Div=15.4;Del=6.7;Ins=2.1;SWscore=13;color=#8686ac'

In [108]:
fa = SeqIO.parse(genome_path, "genbank")

In [109]:
chroms = [g for g in fa]
len(chroms)

24

In [110]:
for c in chroms:
    print(c.id)
    print(c.description)

NC_060925.1
Homo sapiens isolate CHM13 chromosome 1, alternate assembly T2T-CHM13v2.0
NC_060926.1
Homo sapiens isolate CHM13 chromosome 2, alternate assembly T2T-CHM13v2.0
NC_060927.1
Homo sapiens isolate CHM13 chromosome 3, alternate assembly T2T-CHM13v2.0
NC_060928.1
Homo sapiens isolate CHM13 chromosome 4, alternate assembly T2T-CHM13v2.0
NC_060929.1
Homo sapiens isolate CHM13 chromosome 5, alternate assembly T2T-CHM13v2.0
NC_060930.1
Homo sapiens isolate CHM13 chromosome 6, alternate assembly T2T-CHM13v2.0
NC_060931.1
Homo sapiens isolate CHM13 chromosome 7, alternate assembly T2T-CHM13v2.0
NC_060932.1
Homo sapiens isolate CHM13 chromosome 8, alternate assembly T2T-CHM13v2.0
NC_060933.1
Homo sapiens isolate CHM13 chromosome 9, alternate assembly T2T-CHM13v2.0
NC_060934.1
Homo sapiens isolate CHM13 chromosome 10, alternate assembly T2T-CHM13v2.0
NC_060935.1
Homo sapiens isolate CHM13 chromosome 11, alternate assembly T2T-CHM13v2.0
NC_060936.1
Homo sapiens isolate CHM13 chromosome 12

In [111]:
# add repeat masker stuff

In [112]:
Counter(repeat_masker['seqid'])

Counter({'chr1': 488811,
         'chr2': 428846,
         'chr3': 357754,
         'chr4': 323747,
         'chr5': 314203,
         'chr6': 295563,
         'chr7': 286892,
         'chrX': 271645,
         'chr8': 260446,
         'chr12': 256325,
         'chr11': 248068,
         'chr10': 242866,
         'chr9': 227836,
         'chr16': 202330,
         'chr15': 193441,
         'chr13': 179518,
         'chr14': 175789,
         'chr17': 173101,
         'chr19': 136837,
         'chr18': 130108,
         'chr20': 129455,
         'chrY': 103356,
         'chr22': 90318,
         'chr21': 73819,
         'chrM': 5})

In [113]:
for c in chroms:
    chrom_num = c.description.split()[5][:-1]
    curr_repeat_masker = repeat_masker.filter(pl.col('seqid') == f"chr{chrom_num}")
    to_add = []
    for row in tqdm(curr_repeat_masker.iter_rows(named=True)):
        # print(row)
        strand = 1 if row['strand'] == "+" else -1
        name = row['attributes'].split()[0].split("=")[1]
        type_ = row['attributes'].split()[1]

        new_feature = SeqFeature(
            location=FeatureLocation(
                start=row['start'] - 1,  # Convert from 1-indexed to 0-indexed
                end=row['end'],           # End is already correct (exclusive)
                strand=strand
            ),
            type="repeat_region",  # Use standard GenBank feature type
            qualifiers={
                "rpt_type": [name],      # Repeat name (e.g., LTR14A)
                "rpt_family": [type_],   # Repeat family (e.g., LTR/ERVK)
                "note": [f"{name} ({type_})"]  # Human-readable note
            }
        )
        to_add.append(new_feature)
    
    # break
    c.features.extend(to_add)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [114]:
len(chroms)

24

In [118]:
SeqIO.write(chroms, "/large_storage/hielab/changdan/euk_genomes/GCF_009914755.1_T2T-CHM13v2.0_genomic.repeat_masker.gbff", 'genbank')

24

In [115]:
c = chroms[21]
len(c)

51324926

In [116]:
len(c.features)

106117

In [117]:
SeqIO.write(c, "../output/chrom_22.repeat_masker.gb", 'genbank')

1

In [95]:
repeat_masker.filter((pl.col('seqid') == 'chr22') & (pl.col('start') > 31139000) & (pl.col('end') < 31145000))

seqid,source,type,start,end,score,strand,phase,attributes
str,str,str,i64,i64,i64,str,str,str
"""chr22""","""RepeatMasker-4.0.1""","""similarity""",31139004,31139342,2688,"""-""",""".""","""Target=LTR14A LTR/ERVK 1 344;D…"
"""chr22""","""RepeatMasker-4.0.1""","""similarity""",31139351,31139472,825,"""-""",""".""","""Target=HERVK14-int LTR/ERVK 59…"
"""chr22""","""RepeatMasker-4.0.1""","""similarity""",31139473,31139529,338,"""-""",""".""","""Target=MER41C LTR/ERV1 6 61;Di…"
"""chr22""","""RepeatMasker-4.0.1""","""similarity""",31139530,31139607,622,"""-""",""".""","""Target=HERVK14-int LTR/ERVK 58…"
"""chr22""","""RepeatMasker-4.0.1""","""similarity""",31139608,31139666,194,"""+""",""".""","""Target=MIRc SINE/MIR 126 188;D…"
…,…,…,…,…,…,…,…,…
"""chr22""","""RepeatMasker-4.0.1""","""similarity""",31143153,31143497,2821,"""-""",""".""","""Target=LTR14A LTR/ERVK 1 344;D…"
"""chr22""","""RepeatMasker-4.0.1""","""similarity""",31143700,31143984,2274,"""-""",""".""","""Target=AluSc SINE/Alu 1 296;Di…"
"""chr22""","""RepeatMasker-4.0.1""","""similarity""",31143985,31144095,801,"""-""",""".""","""Target=AluSx SINE/Alu 6 116;Di…"


In [101]:
[f for f in c.features if f.type == 'SINE/Alu']

[SeqFeature(SimpleLocation(ExactPosition(5512), ExactPosition(5816), strand=1), type='SINE/Alu', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(9039), ExactPosition(9303), strand=1), type='SINE/Alu', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(12158), ExactPosition(12304), strand=-1), type='SINE/Alu', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(33001), ExactPosition(33304), strand=-1), type='SINE/Alu', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(42167), ExactPosition(42447), strand=-1), type='SINE/Alu', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(43485), ExactPosition(43766), strand=-1), type='SINE/Alu', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(44067), ExactPosition(44375), strand=1), type='SINE/Alu', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(45170), ExactPosition(45476), strand=1), type='SINE/Alu', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(48289), ExactPosition(48591), s